<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/20202/pcd_20202_0901_intro_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Channels!

Un proceso debe escribir y un proceso distinto debe leer. Un canal Síncrono, no puede ser leido y escrito al mismo tiempo por un solo proceso.

In [1]:
%%writefile 1.go
package main

import (
    "fmt"
)

func main() {
    var ch chan string
    ch = make(chan string)

    go func() {
        ch<- "Hola, Mundo canalizado!"
    }()

    str := <-ch
    fmt.Printf("Mensaje: %s\n", str)
}

Writing 1.go


In [4]:
!go run 1.go

Mensaje: Hola, Mundo canalizado!


In [17]:
%%writefile 2.go
package main

import (
    "fmt"
    "math/rand"
    "time"
)

func rng(id, n int, ch chan string) {
    for i := 0; i < n; i++ {
        time.Sleep(time.Millisecond)
        num := rand.Intn(100)
        ch <- fmt.Sprintf("pid=%d\ti=%d\tnumber=%d", id, i, num);
    }
}

func main() {
    rand.Seed(time.Now().UTC().UnixNano())
    var ch chan string
    ch = make(chan string)

    n1, n2 := 11, 14
    go rng(1, n1, ch)
    go rng(2, n2, ch)

    for i := 0; i < n1+n2; i++ {
        fmt.Printf("Mensaje: %s\n", <-ch)
    }
}

Overwriting 2.go


In [18]:
!go run 2.go

Mensaje: pid=1	i=0	number=89
Mensaje: pid=2	i=0	number=18
Mensaje: pid=1	i=1	number=92
Mensaje: pid=2	i=1	number=91
Mensaje: pid=1	i=2	number=47
Mensaje: pid=2	i=2	number=46
Mensaje: pid=1	i=3	number=4
Mensaje: pid=2	i=3	number=63
Mensaje: pid=2	i=4	number=55
Mensaje: pid=1	i=4	number=20
Mensaje: pid=1	i=5	number=84
Mensaje: pid=2	i=5	number=90
Mensaje: pid=2	i=6	number=55
Mensaje: pid=1	i=6	number=47
Mensaje: pid=1	i=7	number=57
Mensaje: pid=2	i=7	number=4
Mensaje: pid=2	i=8	number=48
Mensaje: pid=1	i=8	number=43
Mensaje: pid=1	i=9	number=95
Mensaje: pid=2	i=9	number=2
Mensaje: pid=2	i=10	number=59
Mensaje: pid=1	i=10	number=64
Mensaje: pid=2	i=11	number=72
Mensaje: pid=2	i=12	number=90
Mensaje: pid=2	i=13	number=57


In [27]:
%%writefile 3.go
package main

import (
    "fmt"
    "math/rand"
    "time"
)

func rng(id, n int, ch chan string) {
    for i := 0; i < n; i++ {
        time.Sleep(time.Millisecond)
        num := rand.Intn(100)
        ch <- fmt.Sprintf("pid=%d\ti=%d\tnumber=%d", id, i, num);
    }
    close(ch)
}

func main() {
    rand.Seed(time.Now().UTC().UnixNano())
    var ch1, ch2 chan string
    ch1 = make(chan string)
    ch2 = make(chan string)

    n1, n2 := 11, 14
    go rng(1, n1, ch1)
    go rng(2, n2, ch2)

    go func() {
        for msg := range ch1 {
            fmt.Printf("Mensaje: %s\n", msg)
        }
    }()
    
    for msg := range ch2 {
        fmt.Printf("Mensaje: %s\n", msg)
    }
}

Overwriting 3.go


In [28]:
!go run 3.go

Mensaje: pid=1	i=0	number=70
Mensaje: pid=2	i=0	number=36
Mensaje: pid=1	i=1	number=71
Mensaje: pid=2	i=1	number=28
Mensaje: pid=1	i=2	number=4
Mensaje: pid=2	i=2	number=47
Mensaje: pid=1	i=3	number=18
Mensaje: pid=2	i=3	number=70
Mensaje: pid=1	i=4	number=93
Mensaje: pid=2	i=4	number=82
Mensaje: pid=1	i=5	number=19
Mensaje: pid=2	i=5	number=77
Mensaje: pid=1	i=6	number=71
Mensaje: pid=2	i=6	number=78
Mensaje: pid=1	i=7	number=11
Mensaje: pid=2	i=7	number=25
Mensaje: pid=1	i=8	number=0
Mensaje: pid=2	i=8	number=19
Mensaje: pid=1	i=9	number=8
Mensaje: pid=2	i=9	number=73
Mensaje: pid=1	i=10	number=85
Mensaje: pid=2	i=10	number=74
Mensaje: pid=2	i=11	number=72
Mensaje: pid=2	i=12	number=9
Mensaje: pid=2	i=13	number=92
fatal error: all goroutines are asleep - deadlock!

goroutine 1 [chan receive]:
main.main()
	/content/3.go:33 +0x2b4

goroutine 20 [chan receive]:
main.main.func1(0xc420064060)
	/content/3.go:28 +0xcd
created by main.main
	/content/3.go:27 +0x20b
exit status 2
